# 1 导入库并查看数据

## 1.1 导入库

In [10]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

%matplotlib inline

## 1.2 导入数据

In [13]:
data = pd.read_csv('creditcard.csv')
data.head()

# 2 数据预处理

## 2.1 检查数据分布

In [17]:
# 绘制 Amount 特征的分布图（直方图）
sns.histplot(data['Amount'], bins=50, kde=True)
plt.title('Distribution of Amount')
plt.xlabel('Amount')
plt.ylabel('Frequency')
plt.show()

In [18]:
# 绘制 Amount 特征的箱线图
sns.boxplot(x=data['Amount'])
plt.title('Boxplot of Amount')
plt.xlabel('Amount')
plt.show()

In [19]:
# 绘制 Amount 特征的核密度估计图
sns.kdeplot(data['Amount'], shade=True)
plt.title('KDE Plot of Amount')
plt.xlabel('Amount')
plt.ylabel('Density')
plt.show()

## 2.2 处理不平衡数据

### 2.2.1 使用 SMOTE 进行过采样

In [20]:
# 使用 SMOTE 进行过采样
from imblearn.over_sampling import SMOTE
from collections import Counter

# 分离特征和目标变量
X = data.drop(columns=['Class'])
y = data['Class']

# 应用 SMOTE
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X, y)

# 检查重新采样后的目标变量分布
print(f"Original dataset class distribution: {Counter(y)}")
print(f"Resampled dataset class distribution: {Counter(y_resampled)}")

# 可视化重新采样后的分布
sns.countplot(x=y_resampled)
plt.title('Resampled Class Distribution')
plt.xlabel('Class')
plt.ylabel('Frequency')
plt.show()

### 2.2.2 使用加权损失函数训练模型

In [21]:
# 使用加权损失函数训练模型
from sklearn.linear_model import LogisticRegression
from sklearn.utils.class_weight import compute_class_weight

# 计算类别权重
class_weights = compute_class_weight('balanced', classes=np.unique(y), y=y)
class_weights_dict = {0: class_weights[0], 1: class_weights[1]}
print(f"Class Weights: {class_weights_dict}")

# 初始化逻辑回归模型并设置 class_weight 参数
model = LogisticRegression(class_weight=class_weights_dict, random_state=42)
model.fit(X, y)

# 3 数据可视化

## 3.1 绘制特征分布图

In [22]:
# 绘制直方图
for column in data.columns:
    if column != 'Class':  # 排除目标变量
        plt.figure(figsize=(8, 4))
        sns.histplot(data[column], bins=50, kde=True)
        plt.title(f'Distribution of {column}')
        plt.xlabel(column)
        plt.ylabel('Frequency')
        plt.show()

# 绘制箱线图
for column in data.columns:
    if column != 'Class':  # 排除目标变量
        plt.figure(figsize=(8, 4))
        sns.boxplot(x=data[column])
        plt.title(f'Boxplot of {column}')
        plt.xlabel(column)
        plt.show()

## 3.2 使用相关性热图查看特征之间的关系

In [23]:
# 计算相关性矩阵
correlation_matrix = data.corr()

# 绘制相关性热图
plt.figure(figsize=(12, 8))
sns.heatmap(correlation_matrix, cmap='coolwarm', annot=False, fmt=".2f")
plt.title('Correlation Matrix')
plt.show()

## 3.3 可视化欺诈与非欺诈交易的特征差异

In [24]:
# 按目标变量分组
fraud = data[data['Class'] == 1]
non_fraud = data[data['Class'] == 0]

# 绘制特征分布对比
for column in data.columns:
    if column != 'Class':  # 排除目标变量
        plt.figure(figsize=(8, 4))
        sns.kdeplot(fraud[column], label='Fraud', shade=True, color='red')
        sns.kdeplot(non_fraud[column], label='Non-Fraud', shade=True, color='blue')
        plt.title(f'Distribution of {column} by Class')
        plt.xlabel(column)
        plt.ylabel('Density')
        plt.legend()
        plt.show()

# 4 特征工程

## 4.1 特征缩放

In [25]:
# 对 Amount 特征进行标准化
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
data['Amount'] = scaler.fit_transform(data[['Amount']])

# 如果需要对其他特征进行归一化，可以使用 MinMaxScaler
from sklearn.preprocessing import MinMaxScaler

scaler_minmax = MinMaxScaler()
data_scaled = scaler_minmax.fit_transform(data)

## 4.2 特征选择

### 4.2.1 相关性分析

In [26]:
# 计算相关性矩阵
correlation_matrix = data.corr()

# 绘制热图
plt.figure(figsize=(12, 8))
sns.heatmap(correlation_matrix, cmap='coolwarm', annot=False, fmt=".2f")
plt.title('Correlation Matrix')
plt.show()

# 查看与目标变量 Class 的相关性
correlation_with_target = correlation_matrix['Class'].sort_values(ascending=False)
print("Correlation with target variable (Class):")
print(correlation_with_target)

### 4.2.2 特征重要性

In [ ]:
# 使用随机森林计算特征重要性
from sklearn.ensemble import RandomForestClassifier

# 分离特征和目标变量
X = data.drop(columns=['Class'])
y = data['Class']

# 训练随机森林模型
rf_model = RandomForestClassifier(random_state=42)
rf_model.fit(X, y)

# 获取特征重要性
feature_importances = rf_model.feature_importances_
feature_names = X.columns

# 可视化特征重要性
plt.figure(figsize=(12, 8))
sns.barplot(x=feature_importances, y=feature_names)
plt.title('Feature Importances')
plt.xlabel('Importance')
plt.ylabel('Feature')
plt.show()